In [ ]:
## Command to see whether packages are installed and installs packages if not yet installed
if(!any(rownames(installed.packages()) == 'ggplot2')) install.packages('ggplot2')
if(!any(rownames(installed.packages()) == 'caret')) install.packages('caret')
if(!any(rownames(installed.packages()) == 'rpart')) install.packages('rpart')
if(!any(rownames(installed.packages()) == 'rpart.plot')) install.packages('rpart.plot')
if(!any(rownames(installed.packages()) == 'dplyr')) install.packages('dplyr')
if(!any(rownames(installed.packages()) == 'randomForest')) install.packages('randomForest')
if(!any(rownames(installed.packages()) == 'e1071')) install.packages('e1071')
if(!any(rownames(installed.packages()) == 'purrr')) install.packages('purrr')
if(!any(rownames(installed.packages()) == 'DescTools')) install.packages('DescTools')
if(!any(rownames(installed.packages()) == 'tidyr')) install.packages('tidyr')
if(!any(rownames(installed.packages()) == 'glmnet')) install.packages('glmnet')

library(glmnet)
library(tidyr)
library(DescTools)
library(purrr)
library(ggplot2)
library(caret)
library(rpart)
library(rpart.plot)
library(randomForest)
library(dplyr)
library(e1071)


# Load dataset
### Link to dataset: https://www.kaggle.com/johnsmith88/heart-disease-dataset
### Ben Wijnen, 21-04-2020


In [ ]:
rm(list = ls())  # Delete everything that is in R's memory
options(scipen=999) 
dataset <- read.csv("heart.csv", header = TRUE)


In [ ]:
## Explore dataset
sapply(dataset, function(x) sum(is.na(x))) # map the number of missings
str(dataset)

In [ ]:
## change factors to factors
factor_variables <- c("sex", 
                      "thal", 
                      "target", 
                      "cp", 
                      "exang") 

dataset[factor_variables] <- lapply(dataset[factor_variables], factor) 
                                            

In [ ]:
# Describtive statistics 
summary(dataset) ### Numeric variables are highlighted with *

# map levels and frequencies per categorical variable
temp <-  dataset %>% keep(is.factor)
for (i in 1:ncol(temp)) {
  print(names(temp[i]))
  print(Freq(temp[,i]))}

In [ ]:
## Histrograms per variable to check distribution and outliers
dataset %>%
  keep(is.numeric) %>%
  gather() %>%
  ggplot(aes(value)) +
  facet_wrap(~ key, scales = "free") +
  geom_histogram()

In [ ]:
## Check near-zero variance
nearZeroVar(dataset, saveMetrics = TRUE)


In [ ]:
##### SET Y #######
dataset$Y <- dataset$target ## set Y
dataset$target <- NULL # delete duplicate variable 

In [ ]:
## Create train/test set
set.seed (123456789)
dt = sort(sample(nrow(dataset), nrow(dataset)*.7)) ## 70% in train set
train<-dataset[dt,]
test<-dataset[-dt,]

# Cross-validation
ctrl <- trainControl(method = "repeatedcv",
                     number = 5,    # ten folds
                     repeats = 3)   # repeated three times

In [ ]:
# Run models
## Run LM
lm <- train(Y ~ ., method = "glm", 
            data = train, 
            trControl = ctrl, 
            preProcess = c("center"))
lm
plot(varImp(lm))


In [ ]:
## Run LASSO
LassoFit <- train(Y ~ ., data = train,
                  method = "glmnet",
                  family = "binomial",
                  trControl = ctrl)
LassoFit
coef(LassoFit$finalModel, LassoFit$bestTune$lambda)
plot(varImp(LassoFit,scale=F))



In [ ]:
## Run kNN
knnFit <- train(Y ~ ., 
                data = train, 
                method = "knn", 
                trControl = ctrl, 
                preProcess = c("center","scale"))

knnFit
plot(knnFit)
plot(varImp(knnFit))


In [ ]:
## Run regression tree
tree <- rpart(Y~., data = train, control = rpart.control(minsplit = 15, cp = 0.01))
rpart.plot(tree, main="Y met alle variabelen")

In [ ]:
## Run RF
rf <- train(Y ~ ., method = "rf", 
            data = train, 
            trControl = ctrl, 
            preProcess = c("center"))
rf
plot(rf)
varImpPlot(rf$finalModel)


In [ ]:
## Check performance on test set
pred_lm <- predict(lm, newdata = test)
print("Confusing matrix LM")
confusionMatrix(pred_lm, test$Y)

pred_knn <- predict(knnFit, newdata = test)
print("Confusing matrix kNN")
confusionMatrix(pred_knn, test$Y)

pred_lasso <- predict(LassoFit, newdata = test)
print("Confusing matrix Lasso")
confusionMatrix(pred_lasso, test$Y)

pred_rf <- predict(rf, newdata = test)
print("Confusing matrix Random Forest")
confusionMatrix(pred_rf, test$Y)